In [1]:
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from optimization import *

In [2]:
def sample(d):
    return torch.concat((torch.rand(d), torch.ones(1)))

In [3]:
agent_attempts = 2
agent_iters = 200
dm_attempts = 6
dm_iters = 2000
temperature = 0.02

In [4]:
m = 2
d = 2
u = torch.tensor([0.15, 0.1])
W = torch.tensor([[1., 1/np.sqrt(5)], [0., 2/np.sqrt(5)], [0.2, 0.0]], dtype=torch.float32)
phi_opt = torch.tensor([[1., 0., 0.2], [1/np.sqrt(5), 2/np.sqrt(5), 0.]], dtype=torch.float32)
Xs = []
X_hats = []
phi = torch.rand((m, d + 1))
x = sample(d)
x_hat = gragent(x, phi, u, attempts=agent_attempts, iters=agent_iters, temp=temperature)
Xs.append(x)
X_hats.append(x_hat)
dataset = torch.stack(Xs)
dataset_hat = torch.stack(X_hats)
cum_dm_reward = dm_reward(dataset @ W, dataset_hat, phi, temp=temperature)
opt_cum_dm_reward = dm_reward(dataset @ W, dataset_hat, phi_opt, temp=temperature)

C:\Users\Alex\Desktop\academics\strat-bandits\strategic-bandits\optimization.py:93: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matricesor `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:2318.)
  return softmax( phi @ x.T , temp=temp)


In [ ]:
phi = gr_phi(dataset_hat, dataset @ W, u, temp=temperature, attempts=dm_attempts, iters=dm_iters)
x = sample(d)
x_hat = gragent(x, phi, u, attempts=agent_attempts, iters=agent_iters, temp=temperature)
Xs.append(x)
X_hats.append(x_hat)
dataset = torch.stack(Xs)
dataset_hat = torch.stack(X_hats)
new_dm_reward = dm_reward(dataset @ W, dataset_hat, phi, temp=temperature)
cum_dm_reward += new_dm_reward

opt_new_dm_reward = dm_reward(dataset @ W, dataset_hat, phi_opt, temp=temperature)
opt_cum_dm_reward += opt_new_dm_reward

lbls = [a[0] > a[1] for a in classify(x=dataset, phi=phi_opt, temp=temperature).T]
preds = [a[0] > a[1] for a in classify(x=dataset_hat, phi=phi, temp=temperature).T]
print('Predicted:', cum_dm_reward.item(), new_dm_reward.item())
print('Optimal:  ', opt_cum_dm_reward.item(), opt_new_dm_reward.item())

 61%|██████████████████████████████████████████████▎                             | 1220/2000 [00:00<00:00, 2019.01it/s]

In [ ]:
plt.gca().set_aspect('equal')
plt.scatter(dataset_hat[:,0], dataset_hat[:,1], c=preds)

In [ ]:
plt.gca().set_aspect('equal')
plt.scatter(dataset[:,0], dataset[:,1], c=lbls)